In [ ]:
import numpy as np
import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from models.models import *
from utils.utils import *
from utils.dataset import *

In [ ]:
BATCH_SIZE = 512
LEARNING_RATE = 0.01
EPOCHS = 1
RANDOM_SEED = 42
WEIGHT_DECAY = 1e-6
MODEL_PATH = 'best/fm_full_tra.pth'
LOG_PATH = 'log/fm_full_tra.csv'
DATA_PATH = 'data/'

torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
events, users, items = load_data(DATA_PATH)
train_df, valid_df = train_test_split(events, test_size=0.2, random_state=RANDOM_SEED)
train_mat = get_csr_mat(train_df, users.shape[0], items.shape[0])
valid_mat = get_csr_mat(events, users.shape[0], items.shape[0])
train_data = BPRData(train_df, train_mat, users, items, num_neg=1, device=device, add_weight=True)
valid_data = BPRData(valid_df, valid_mat, users, items, num_neg=1, device=device, add_weight=True)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
sparse_dims, dense_dim = train_data.get_dims()
model = FM(sparse_dims, dense_dim, embed_dim=32).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [ ]:
with open(LOG_PATH, 'w') as f:
    f.write('train_loss,valid_loss,train_precision,valid_precision,roc_auc_train,roc_auc_valid\n')
p_best = 0
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_data.neg_sample()
    progress = tqdm(train_loader, desc=f'Epoch {epoch + 1}', disable=False)
    for x_pos_sparse, x_pos_dense, x_neg_sparse, x_neg_dense, weight in progress:
        optimizer.zero_grad()
        pos_score = model(x_pos_sparse, x_pos_dense)
        neg_score = model(x_neg_sparse, x_neg_dense)
        loss = -(torch.log(torch.sigmoid(pos_score - neg_score)) * weight).mean()
        if torch.isinf(loss): continue
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    model.eval()
    valid_loss = 0
    valid_data.neg_sample()
    for x_pos_sparse, x_pos_dense, x_neg_sparse, x_neg_dense, weight in valid_loader:
        pos_score = model(x_pos_sparse, x_pos_dense)
        neg_score = model(x_neg_sparse, x_neg_dense)
        loss = -(torch.log(torch.sigmoid(pos_score - neg_score)) * weight).mean()
        valid_loss += loss.item()
    valid_loss /= len(valid_loader)

    p_train, p_valid, roc_train, roc_valid = \
        metrics_at_k(model, valid_df, train_df, users, items, k=10, device=device, precision=True, roc_auc=True)
    if p_valid > p_best:
        p_best = p_valid
        torch.save(model.state_dict(), MODEL_PATH)
    with open(LOG_PATH, 'a') as f:
        f.write(f'{train_loss:.4f},{valid_loss:.4f},{p_train:.4f},{p_valid:.4f},{roc_train:.4f},{roc_valid:.4f}\n')
        
    print(f'Train Loss: {train_loss:.4f}, ' +
          f'Valid Loss: {valid_loss:.4f}, ' +
          f'Precision@10 (Train): {p_train:.4f}, ' +
          f'Precision@10 (Valid): {p_valid:.4f}, ' +
          f'ROC AUC (Train): {roc_train:.4f},' +
          f'ROC AUC (Valid): {roc_valid:.4f}')